In [67]:
# Notebook to save the time profile of events for reading into simulatoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import math 

In [68]:
Mode = ["CRAB","CRAB"]

data = pd.read_csv(f"../Files/{Mode[0]}/EventInfo_merge_{Mode[1]}.csv", delimiter = ",", names = ["event", "x", "y", "z", "t"])

# Cut the first bin
data = data[data['t'] > 200]
data = data.reset_index(drop=True)
display(data)

,event,x,y,z,t
0,11,0.144,-0.233,-10.656,14417.8
1,11,0.142,-0.233,-10.662,14442.2
2,11,0.138,-0.230,-10.668,14479.6
3,11,0.140,-0.230,-10.679,14530.7
4,11,0.139,-0.233,-10.681,14538.9
...,...,...,...,...,...
9145169,10000,0.007,0.185,-11.342,18198.7
9145170,10000,0.007,0.186,-11.342,18199.9
9145171,10000,0.006,0.186,-11.343,18202.5
9145172,10000,0.006,0.186,-11.342,18203.0


In [69]:

# Here we subtract off the first time bin to shift all the EL values to zero

def GetSubDf(data, var, var_sub, query):
    data_copy = data.copy(deep=True)

    data_sub = data_copy.drop_duplicates(subset= ["event"], keep='first')
    data_sub = data_sub.drop(columns = query)

    data_copy['Count'] = 1
    data_copy = data_copy.groupby(["event"]).Count.count().reset_index()

    # get a dataframe with the first time row to subtract from
    data_merge = pd.DataFrame()
    data_sub = data_sub.merge(data_copy, on='event', how="inner")
    data_sub = data_sub.reset_index(drop = True)
    data_sub = data_sub.loc[data_sub.index.repeat(data_sub.Count)]
    data_sub = data_sub.drop(columns = ["Count"])
    data_sub = data_sub.rename(columns={var: var_sub})
    data_sub = data_sub.reset_index(drop=True)
    return data_sub



In [70]:

data_sub = pd.DataFrame()
query = ["x", "y", "z"]
data_sub = GetSubDf(data, "t", "t_sub", query)
data["t"] = data["t"] - data_sub["t_sub"]

data_sub = pd.DataFrame()
query = ["t", "y", "z"]
data_sub = GetSubDf(data, "x", "x_sub", query)
data["x"] = data["x"] - data_sub["x_sub"]

data_sub = pd.DataFrame()
query = ["x", "t", "z"]
data_sub = GetSubDf(data, "y", "y_sub", query)
data["y"] = data["y"] - data_sub["y_sub"]

data_sub = pd.DataFrame()
query = ["x", "y", "t"]
data_sub = GetSubDf(data, "z", "z_sub", query)
data["z"] = data["z"] - data_sub["z_sub"]

display(data)


,event,x,y,z,t
0,11,0.000,0.000,0.000,0.0
1,11,-0.002,0.000,-0.006,24.4
2,11,-0.006,0.003,-0.012,61.8
3,11,-0.004,0.003,-0.023,112.9
4,11,-0.005,0.000,-0.025,121.1
...,...,...,...,...,...
9145169,10000,-0.144,-0.065,-0.702,3605.5
9145170,10000,-0.144,-0.064,-0.702,3606.7
9145171,10000,-0.145,-0.064,-0.703,3609.3
9145172,10000,-0.145,-0.064,-0.702,3609.8


In [71]:
# Export the dataframe 
data = data[ data["event"] < 5000]

# Convert some columns to mm to save space
data.to_csv("../Files/CRAB/CRAB_Profiles.csv", index=False, header=False, float_format='%4.2f')